In [2]:
import pandas as pd
import asyncio
import aiohttp
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import sys
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from tqdm.auto import tqdm
import random
import json

In [97]:
API_KEY = "sk-live-a5OIPGO3CpO5onxTLyLRJihpvUOW0bqmBOHEFrvF"
USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"

In [98]:
def get_all_news(i):
    headers = {
        "User-Agent": USER_AGENT,
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
    }
    url = f"https://timesofindia.indiatimes.com/archivelist/starttime-{i}.cms"
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)

    time.sleep(random.randint(5, 10))

    content = driver.page_source
    driver.quit()
    
    soup = BeautifulSoup(content, 'html.parser')
    soup = soup.find_all('td')
    soup = soup[4:6]
    link_headlines_tuples = []
    for td in soup:
        a_tags = td.find_all('a')
        for a in a_tags:
            link = ""
            if "https" in a['href'] or "http" in a['href']:
                link = a['href']
            else:
                link = f"https://timesofindia.indiatimes.com{a['href']}"
            headline = a.text
            link_headlines_tuples.append((link, headline))
    return link_headlines_tuples

In [99]:
def fetch_tags(link):
    splits = link.split("/")
    tags = splits[3:-3]
    return tags, splits[-1].replace(".cms", "")

In [100]:
def fetch(url, headline, df=None):
    headers = {
        "User-Agent": USER_AGENT,
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
    }
    try:
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        driver = webdriver.Chrome(options=chrome_options)
        driver.get(url)
    
        time.sleep(random.randint(15, 20))
    
        content = driver.page_source
        driver.quit()
        soup = BeautifulSoup(content, 'html.parser')
        div_content = soup.find('div', class_='_s30J clearfix')
        date_time = soup.find('div', class_="xf8Pm byline").text
        date_time = date_time.split('/')[2]

        if div_content:
            content = div_content
            content = content.text
            tags, article_number = fetch_tags(url)
            try:
                # add_news_article_to_dynamodb(url, headline, content, tags, article_number, date)
                if df is not None:
                    df.loc[len(df)] = [headline, url, content, date_time, tags, article_number]
                return True
            except Exception as e:
                print(f"Unable to add data to Dynamo DB for date: {date} with error: {e}")
                sys.exit(1)
        else:
            print("Div with the specified class not found.")
            
    except Exception as e:
        print(f"An error occurred for {url}: {e}")
        return None

In [101]:
def fetch_all(link_headlines_tuples, df=None):
    for (link, headline) in tqdm(link_headlines_tuples):
        fetch(link, headline, df)

In [94]:
def scrap_news(page_suffix, df=None):
    
    # update pagesuffix-1, currently it just fetches for one day
    for i in range(page_suffix, page_suffix-1, -1):
        start_time  = time.time()
        print(f"Starting scraping with suffix: {i}")
        try:
            link_headlines_tuples = get_all_news(i)
        except Exception as e:
            print(f"Exception caught while calling get_all_news with error: {e}")
            
        fetch_all(link_headlines_tuples, df)

        
        print("Done for date")
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"Time taken to run the function: {elapsed_time:.4f} seconds")
        print("\n\n")

In [ ]:
# news_articles_for_day = pd.DataFrame(columns=["headline", "url", "content", "date_time", "tags", "article_number"])
# scrap_news(45594, news_articles_for_day)
# news_articles_for_day.to_csv("./fetched-data/news_articles_for_today.csv", index=False)

In [3]:
news_articles_for_day = pd.read_csv("./fetched-data/news_articles_for_today.csv")

In [37]:
all_news = news_articles_for_day.content.values

In [175]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(
    template="""You will be provided with news and a comma separated list of different sectors in which Indian stock market can be divided. \
You need to go through the news and identify the sectors that the news can impact. Only include the sectors that are directly influenced \
by the news. If there is a neutral impact by the news dont include that in the response. Only include the sector in the output if there \
is a strong correlation between the sector and the news. Do not try to invent any new sector and choose the sector only from the comma \
separated list that I will provide. If there are no sectors that are impacted, dont return anything. For each sector that the news can \
impact, tell whether the impact would be postive or negative. Also give a brief summary behind your reasoning. 

Only return a list of json with the following keys: sector_name, impact and reasoning.

I'm adding an example for your reference:

[
{{
    "sector_name": "ABC",
    "impact": "ABC",
    "reasoning: "ABC"
}}
]


News article: {news}
Sectors: {sectors}
""",
    input_variables=["news", "sectors"],
)

In [176]:
prompt

PromptTemplate(input_variables=['news', 'sectors'], input_types={}, partial_variables={}, template='You will be provided with news and a comma separated list of different sectors in which Indian stock market can be divided. You need to go through the news and identify the sectors that the news can impact. Only include the sectors that are directly influenced by the news. If there is a neutral impact by the news dont include that in the response. Only include the sector in the output if there is a strong correlation between the sector and the news. Do not try to invent any new sector and choose the sector only from the comma separated list that I will provide. If there are no sectors that are impacted, dont return anything. For each sector that the news can impact, tell whether the impact would be postive or negative. Also give a brief summary behind your reasoning. \n\nOnly return a list of json with the following keys: sector_name, impact and reasoning.\n\nI\'m adding an example for y

In [177]:
from langchain_ollama import OllamaLLM
from langchain.chains import LLMChain

In [178]:
ollama = OllamaLLM(model="llama3.1:latest")
llm_chain = prompt | ollama

In [179]:
output = llm_chain.invoke({"news": news, "sectors": all_sectors })

In [181]:
print(output)

Based on the news article and the sectors provided, here is the list of impacted sectors with their respective impact and reasoning:

[
  {
    "sector_name": "finance_stock_broking",
    "impact": "Negative",
    "reasoning": "The news about opposition unity in Maharashtra polls may lead to market volatility and uncertainty, which could negatively impact stock prices and trading activities."
  },
  {
    "sector_name": "bank_private",
    "impact": "Negative",
    "reasoning": "As the news suggests a united opposition, it may lead to economic instability and potentially negative impacts on private banks, especially if the BJP-led government is not returned to power."
  },
  {
    "sector_name": "finance_asset_management",
    "impact": "Negative",
    "reasoning": "The potential change in government may lead to a reevaluation of investment strategies, which could negatively impact asset management companies and their clients."
  }
]

Note that these are the only sectors directly impac

In [41]:
news_articles_for_day[news_articles_for_day['tags'].str.contains(r'\bmovies\b', case=False, na=False)]

,headline,url,content,date_time,tags,article_number
93,Every film I am part of helps me grow as an ar...,https://timesofindia.indiatimes.com/entertainm...,Rukmini Vasanth Actor Rukmini Vasanth is shar...,"Oct 29, 2024, 00:00 IST","['entertainment', 'kannada', 'movies', 'news']",114687715
250,Suriya says he moved to Mumbai to acknowledge ...,https://timesofindia.indiatimes.com/entertainm...,"Kollywood’s most celebrated couple, Suriya a...","Updated: Oct 31, 2024, 06:53 IST","['entertainment', 'tamil', 'movies', 'news']",114705074
297,Did you know Prabhas rejected THIS role in Dee...,https://timesofindia.indiatimes.com/entertainm...,"Rebel star Prabhas, known for his pan-Indian...","Updated: Oct 29, 2024, 01:27 IST","['entertainment', 'telugu', 'movies', 'news']",114705165


In [42]:
news1 = news_articles_for_day.iloc[93].content

In [188]:
output = llm_chain.invoke({"news": news1, "sectors": all_sectors })
print(output)

Here is the output for the given news article:

[
  {
    "sector_name": "FMCG",
    "impact": "Negative",
    "reasoning": "The finance ministry's monthly economic review for Sept said that sluggish urban sales due to stubborn food inflation, while rural demand appeared to be improving."
  },
  {
    "sector_name": "Auto",
    "impact": "Negative",
    "reasoning": "The report cited slowing auto sales in urban centres."
  },
  {
    "sector_name": "Housing",
    "impact": "Negative",
    "reasoning": "The report cited slowing housing sales in urban centres."
  },
  {
    "sector_name": "Industrial Gases and Fuels",
    "impact": "Negative",
    "reasoning": "The country's industrial output contracted for first time in 22 months in Aug, dragged down by a high base effect and decline in mining and electricity and a weak manufacturing sector."
  }
]

Note: I considered only the sectors that have a direct impact on the news article. For example, FMCG (Fast Moving Consumer Goods) is affect

In [5]:
from pydantic import BaseModel
from typing import List

In [43]:
class SectorImpacted(BaseModel):
    sector_name: str
    sentiment: str
    reasoning: str

class ListOfImpactedSector(BaseModel):
    list_of_sector: List[SectorImpacted]

In [44]:
from llama_index.llms.ollama import Ollama
from llama_index.core.program import LLMTextCompletionProgram

In [45]:
llm = Ollama(model="llama3.1:latest", request_timeout=600)

In [46]:
prompt_template_str = """You will be provided with news and a comma separated list of different sectors in which Indian stock market can be divided. \
You need to go through the news and identify the sectors that the news can impact. Only include the sectors that are directly influenced \
by the news. If there is a neutral impact by the news dont include that in the response. Only include the sector in the output if there \
is a strong correlation between the sector and the news. Do not try to invent any new sector and choose the sector only from the comma \
separated list that I will provide. If there are no sectors that are impacted, dont return anything. For each sector that the news can \
impact, tell whether the impact would be postive or negative. Also give a brief summary behind your reasoning. 

News article: {news}
Sectors: {sectors}

Provide the results in list of JSON format, matching the structure of the ListOfImpactedSector model:
- sector_name: the name of the sector impacted
- sentiment: the sentiment towards that sector
- reasoning: the reasoning behind the sentiment towards the sector


"""

In [47]:
program = LLMTextCompletionProgram.from_defaults(
    llm=llm,
    output_cls=ListOfImpactedSector,
    prompt_template_str=prompt_template_str,
    verbose=True,
)

In [48]:
output = program(news = news1, sectors = all_sectors)

In [49]:
output.json()

'{"list_of_sector":[{"sector_name":"film_production_distribution_entertainment","sentiment":"positive","reasoning":"The news article is about an actress, Rukmini Vasanth, sharing her experiences and excitement about working in the film industry. This indicates a positive sentiment towards the sector."}]}'

In [50]:
json.loads(output.json())

{'list_of_sector': [{'sector_name': 'film_production_distribution_entertainment',
   'sentiment': 'positive',
   'reasoning': 'The news article is about an actress, Rukmini Vasanth, sharing her experiences and excitement about working in the film industry. This indicates a positive sentiment towards the sector.'}]}

In [13]:
all_sectors = sectors.sector.values
all_sectors = ", ".join(list(all_sectors))

In [13]:
url = "https://timesofindia.indiatimes.com/entertainment/hindi/bollywood/news/abhishek-kapoor-celebrates-diwali-with-rasha-thadani-and-aaman-devgn-ahead-of-their-untitled-debut-film/articleshow/114705104.cms"
response = requests.get(url)
response.encoding = 'utf-8'
content = response.text

soup = BeautifulSoup(content, 'html.parser')
span = soup.find_all('div', class_="xf8Pm byline")
span[0].text

'TOI Entertainment Desk / etimes.in /  Oct 29, 2024, 01:19 IST'

In [12]:
sectors = pd.read_csv("./fetched-data/stock-sectors.csv")

In [3]:
sectors

,sector,description,total_companies,link
0,abrasives,The one that scraps away all inconsistencies,3,/abrasives
1,advertising_media,The one that creates and distributes media con...,11,/advertising-media
2,agriculture,The one that gives us vegetables and fruits to...,49,/agriculture
3,air_conditioners,The one thats keeps it cool with no noise,6,/air-conditioners
4,airlines,The one that takes us around the world in the ...,6,/airlines
...,...,...,...,...
129,travel_services,The one that takes you to your favourite holidays,17,/travel-services
130,tv_broadcasting_software_production,The one that brings you your favourite TV shows,32,/tv-broadcasting-software-production
131,tyres_allied,The one that makes vehicles move,12,/tyres-allied
132,watches_accessories,The one that makes your life more functional,2,/watches-accessories


In [ ]:
from langchain_community import